<a href="https://colab.research.google.com/github/swagatskalita092/Flask-API/blob/main/FlaskImp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask
!pip install flask_sqlalchemy
!pip install joblib
!pip install nltk


In [2]:
!pip install pyngrok
!pip install flask-ngrok
!ngrok config add-authtoken 2roCSQYQS3Q9iZsAQ8woEaITLcx_5EKEwNKJ4oFZFG4ZyBjPh


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from flask_ngrok import run_with_ngrok
import joblib
import nltk
import re
import string
from pyngrok import ngrok

In [4]:
# Load trained Random Forest model
model_path = "/content/random_forest_issue_classifier.pkl"
model = joblib.load(model_path)

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Enable ngrok

# Database Configuration (SQLite)
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///predictions.db"
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False
db = SQLAlchemy(app)

In [5]:
# Define database model
class Prediction(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    issue_text = db.Column(db.String(500), nullable=False)
    predicted_label = db.Column(db.String(100), nullable=False)
    correct_label = db.Column(db.String(100), nullable=True)

# Create database
with app.app_context():
    db.create_all()

In [6]:
# Root endpoint to check if API is running
@app.route("/")
def home():
    return "Flask API is running with ngrok!"

# ✅ **Text Preprocessing Function**
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = text.strip()  # Remove extra spaces
    return text

In [7]:
# ✅ **Predict Endpoint**
@app.route('/api/predict', methods=['POST'])
def predict():
    try:
        data = request.json
        issue_text = data.get("text")

        # Preprocess the text
        processed_text = preprocess_text(issue_text)

        # Make a prediction
        predicted_label = model.predict([processed_text])[0]

        # Store prediction in the database
        new_prediction = Prediction(issue_text=issue_text, predicted_label=predicted_label)
        db.session.add(new_prediction)
        db.session.commit()

        return jsonify({"id": new_prediction.id, "predicted_label": predicted_label})

    except Exception as e:
        return jsonify({"error": str(e)})

In [8]:
# ✅ **Correct Endpoint**
@app.route('/api/correct', methods=['POST'])
def correct():
    try:
        data = request.json
        prediction_id = data.get("id")
        correct_label = data.get("correct_label")

        # Find the prediction entry in the database
        prediction_entry = Prediction.query.get(prediction_id)

        if not prediction_entry:
            return jsonify({"error": "Prediction ID not found"}), 404

        # Update the correct label
        prediction_entry.correct_label = correct_label
        db.session.commit()

        return jsonify({"message": "Correction saved successfully"})

    except Exception as e:
        return jsonify({"error": str(e)})



In [ ]:
if __name__ == "__main__":
    app.run()